In [13]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension()

import hvplot.pandas

In [28]:
df = pd.read_csv('Derived/All-Journals-Cleaned.csv')

In [29]:
df

,Title,Issue,Journal,Abstract,Link,Author1,Author2,Author3,Author4,Author5,...,Author15,Author16,Author17,JEL1,JEL2,JEL3,JEL4,JEL5,JEL6,JEL7
0,The Economics of the Public Option: Evidence f...,"VOL. 114, NO. 3, MARCH 2024",AMERICAN ECONOMIC REVIEW,We study the effects of competition by state-o...,https://www.aeaweb.org/articles?id=10.1257/aer...,Juan Pablo Atal,José Ignacio Cuesta,Felipe González,Cristóbal Otero,NaN,...,NaN,NaN,NaN,D22,I18,L32,L65,O14,NaN,NaN
1,The Effect of Macroeconomic Uncertainty on Hou...,"VOL. 114, NO. 3, MARCH 2024",AMERICAN ECONOMIC REVIEW,We use randomized treatments that provide diff...,https://www.aeaweb.org/articles?id=10.1257/aer...,Olivier Coibion,Dimitris Georgarakos,Yuriy Gorodnichenko,Geoff Kenny,Michael Weber,...,NaN,NaN,NaN,D12,D81,D84,E21,E23,G51,NaN
2,Optimal Inference for Spot Regressions,"VOL. 114, NO. 3, MARCH 2024",AMERICAN ECONOMIC REVIEW,Betas from return regressions are commonly use...,https://www.aeaweb.org/articles?id=10.1257/aer...,Tim Bollerslev,Jia Li,Yuexuan Ren,NaN,NaN,...,NaN,NaN,NaN,C22,C58,G12,G23,NaN,NaN,NaN
3,The Comparative Statics of Sorting,"VOL. 114, NO. 3, MARCH 2024",AMERICAN ECONOMIC REVIEW,We create a general and tractable theory of in...,https://www.aeaweb.org/articles?id=10.1257/aer...,Axel Anderson,Lones Smith,NaN,NaN,NaN,...,NaN,NaN,NaN,C78,D21,D82,D86,J12,NaN,NaN
4,Mental Models and Learning: The Case of Base-R...,"VOL. 114, NO. 3, MARCH 2024",AMERICAN ECONOMIC REVIEW,We experimentally document persistence of subo...,https://www.aeaweb.org/articles?id=10.1257/aer...,Ignacio Esponda,Emanuel Vespa,Sevgi Yuksel,NaN,NaN,...,NaN,NaN,NaN,D83,D91,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7369,"Like Father, Like Son: Social Network External...","VOL. 1, NO. 1, FEBRUARY 2009",AMERICAN ECONOMIC JOURNAL: MICROECONOMICS,We build an overlapping generations model wher...,https://www.aeaweb.org/articles?id=10.1257/mic...,Antoni Calvó-Armengol,Matthew O. Jackson,NaN,NaN,NaN,...,NaN,NaN,NaN,J12,J13,Z13,NaN,NaN,NaN,NaN
7370,The Compromise Game: Two-Sided Adverse Selecti...,"VOL. 1, NO. 1, FEBRUARY 2009",AMERICAN ECONOMIC JOURNAL: MICROECONOMICS,We analyze a game of two-sided private informa...,https://www.aeaweb.org/articles?id=10.1257/mic...,Juan D. Carrillo,Thomas R. Palfre,NaN,NaN,NaN,...,NaN,NaN,NaN,C91,D82,NaN,NaN,NaN,NaN,NaN
7371,Veto Constraint in Mechanism Design: Inefficie...,"VOL. 1, NO. 1, FEBRUARY 2009",AMERICAN ECONOMIC JOURNAL: MICROECONOMICS,We consider bargaining problems in which parti...,https://www.aeaweb.org/articles?id=10.1257/mic...,Olivier Compte,Philippe Jehiel,NaN,NaN,NaN,...,NaN,NaN,NaN,C78,D82,NaN,NaN,NaN,NaN,NaN
7372,The Origin of the Winner's Curse: A Laboratory...,"VOL. 1, NO. 1, FEBRUARY 2009",AMERICAN ECONOMIC JOURNAL: MICROECONOMICS,The Winner's Curse (WC) is a robust and persis...,https://www.aeaweb.org/articles?id=10.1257/mic...,Gary Charness,Dan Levin,NaN,NaN,NaN,...,NaN,NaN,NaN,D81,D82,NaN,NaN,NaN,NaN,NaN


In [36]:
# Data Manipulations for Later Use
# create a year column
df["Year"] = (df['Issue'].str[-4:]).astype(int)

# identify author and JEL columns
author_columns = [col for col in df.columns if col.startswith("Author")]
jel_columns = [col for col in df.columns if col.startswith("JEL")]

In [37]:
# Make DataFrame Pipline Interactive
dfi = df.interactive()

In [38]:
# Define Panel Widgets

# 1. Journal Dropdown
journal_names = df['Journal'].unique()
journal_names = list(journal_names)
journal_select = pn.widgets.Select(name='Choose Journal:', options=journal_names)

# 2. Year Slider
journal_start_yr = {
    'AMERICAN ECONOMIC REVIEW' : 1999,
    'AMERICAN ECONOMIC REVIEW: INSIGHTS' : 2019,
    'AMERICAN ECONOMIC JOURNAL: APPLIED ECONOMICS' : 2009,
    'AMERICAN ECONOMIC JOURNAL: ECONOMIC POLICY' : 2009,
    'AMERICAN ECONOMIC JOURNAL: MACROECONOMICS' : 2009,
    'AMERICAN ECONOMIC JOURNAL: MICROECONOMICS' : 2009
}
    
year_slider = pn.widgets.IntSlider(name = "Start Year:", start = 1999, end = 2024, step = 1)

# Function to update the slider range based on the journal selected
def update_year_range(event):
    journal_name = event.new
    year_slider.start = journal_start_yr[journal_name]
    year_slider.value = journal_start_yr[journal_name]

# Watch for changes to the choice of journal
journal_select.param.watch(update_year_range, 'value')

# Layout the widgets
layout = pn.Column(journal_select, year_slider)

# Display the layout
layout.show()

Launching server at http://localhost:61671


In [ ]:
# Top Economists in each Journal
# function to get top 10 authors in the selected journal from the selected start year
def get_top_authors(start_year, journal):
    filtered_data = df[(df['Year'] >= start_year) & (df['Journal'] == journal)]
    author_counts = filtered_data['Author'].value_counts().head(10)
    return author_counts.to_frame('Number of Papers')

In [41]:
# test
filtered_data = df[(df['Year'] >= 2009) & (df['Journal'] == "AMERICAN ECONOMIC REVIEW")]
author_counts1 = filtered_data['Author1'].value_counts()
author_counts2 = filtered_data['Author2'].value_counts()
print(author_counts1)
print(author_counts2)


Author1
Daron Acemoglu     17
Raj Chetty         10
Nicholas Bloom      9
Nava Ashraf         7
David Card          7
                   ..
Sergiu Hart         1
Rodrigo Adao        1
Battal Doğan        1
Alejandro Ponce     1
Dan Ariely          1
Name: count, Length: 2031, dtype: int64
Author2
Amy Finkelstein       9
Jonathan Gruber       8
John A. List          7
Michael Greenstone    7
Parag A. Pathak       6
                     ..
Jean-Marc Robin       1
Leonardo Melosi       1
Gary Gorton           1
Amanda Bayer          1
Anat Bracha           1
Name: count, Length: 1710, dtype: int64
